dict_keys(['factors', 'effect_details', 'effect_type'])

In [1]:
import transformers

In [14]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('../data_collection/reddit_ed_0.6.0.sqlite3')

cursor = conn.cursor()

query = f"select * from subreddit_submission_metadata"

df = pd.read_sql_query(query,con=conn)
df["num_words"] = df["selftext"].apply(lambda x: len(x.split()))
word_length_cutoff = 20 # set a very low threshold
print(f"Number of short posts: {len(df[df['num_words'] < word_length_cutoff])}")
filtered_df = df[df["num_words"] >= word_length_cutoff]
print(f"Number of filtered posts (by word length): {len(filtered_df)}")
filtered_df["link_flair_text"] = filtered_df["link_flair_text"].apply(lambda x: x.lower().strip() if x is not None else "")
positive_flags = ["story", "progress"]
negative_flags = ["announcement", "educational", "research request", "resources", "advertise"]
uncertain_flags = ["question", "support", "advice"]
import re
positive_flag_regex = ".*(" + ")|(".join(positive_flags) + ").*"
negative_flag_regex = ".*(" + ")|(".join(negative_flags) + ").*"
uncertain_flag_regex = ".*(" + ")|(".join(uncertain_flags) + ").*"
filtered_df["narrative_positive"] = filtered_df["link_flair_text"].apply(lambda x: 1 if re.match(positive_flag_regex, x) else 0)
filtered_df["narrative_negative"] = filtered_df["link_flair_text"].apply(lambda x: 1 if re.match(negative_flag_regex, x) else 0)
filtered_df["narrative_uncertain"] = filtered_df["link_flair_text"].apply(lambda x: 1 if re.match(uncertain_flag_regex, x) else 0)

Number of short posts: 8940
Number of filtered posts (by word length): 15706


/var/folders/32/km91km0x2s98wl2f_rb4gb0r0000gn/T/ipykernel_51395/4223111357.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["link_flair_text"] = filtered_df["link_flair_text"].apply(lambda x: x.lower().strip() if x is not None else "")
/var/folders/32/km91km0x2s98wl2f_rb4gb0r0000gn/T/ipykernel_51395/4223111357.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["narrative_positive"] = filtered_df["link_flair_text"].apply(lambda x: 1 if re.match(positive_flag_regex, x) else 0)
/

In [15]:
def get_analysis_info(data: pd.DataFrame, type: str):
    column_name = f"narrative_{type}" 
    print(f"Number of {type}: {sum(data[column_name])} / {len(data)}")
    print(f"Values of flair flags: {data[data[column_name] == 1]['link_flair_text'].unique()}")
    return data[data[column_name] == 1][["title","link_flair_text", "selftext"]]

positive_df = get_analysis_info(filtered_df, "positive")

Number of positive: 159 / 15706
Values of flair flags: ['recovery story' 'story time' ':karma:personal story:karma:' 'my story'
 'progress']


,Unnamed: 0,subreddit,selftext,author_fullname,title,score,link_flair_css_class,link_flair_text,author_flair_type,over_18,...,author_flair_css_class,permalink,url,created_utc,num_words,narrative_positive,narrative_negative,narrative_uncertain,narrative_label,predicted_narrative_label
0,0,EatingDisorders,23 F. I just started treatment for an eating d...,NaN,I need moral support,1,NaN,seeking support,NaN,NaN,...,NaN,/r/EatingDisorders/comments/10poqxu/i_need_mor...,https://www.reddit.com/r/EatingDisorders/comme...,1675140001,191,0,0,0,-1,1
1,1,EatingDisorders,I 26 f told the truth for the first time about...,NaN,Opening up about my ed,1,NaN,seeking support,NaN,NaN,...,NaN,/r/EatingDisorders/comments/10pnoxo/opening_up...,https://www.reddit.com/r/EatingDisorders/comme...,1675137504,273,0,0,0,-1,1
2,4,EatingDisorders,Idk who to talk to about this so I resort to r...,NaN,Just ranting ig,1,NaN,NaN,NaN,NaN,...,NaN,/r/EatingDisorders/comments/10pjinj/just_ranti...,https://www.reddit.com/r/EatingDisorders/comme...,1675126245,133,0,0,0,-1,1
3,5,EatingDisorders,I don’t know where to start.. Due to my upbrin...,NaN,Is true recovery possible?,1,NaN,seeking support,NaN,NaN,...,NaN,/r/EatingDisorders/comments/10pj90n/is_true_re...,https://www.reddit.com/r/EatingDisorders/comme...,1675125516,132,0,0,0,-1,1
4,8,EatingDisorders,i abused laxatives/purging/starving and now ev...,NaN,i get “chronic” hiccups due to bulimia (take t...,1,NaN,NaN,NaN,NaN,...,NaN,/r/EatingDisorders/comments/10perl4/i_get_chro...,https://www.reddit.com/r/EatingDisorders/comme...,1675114462,171,0,0,0,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7671,24609,bingeeating,I saw something like this posted on another we...,NaN,You might have a binge eating disorder if...,1,NaN,NaN,NaN,NaN,...,NaN,/r/bingeeating/comments/4iqjys/you_might_have_...,https://www.reddit.com/r/bingeeating/comments/...,1462903827,310,0,0,0,-1,1
7672,24617,PurgingDisorder,"hi,\n\nthanks for making this space.\n\ni was ...",NaN,hi,6,NaN,NaN,NaN,NaN,...,NaN,/r/PurgingDisorder/comments/pdq9pz/hi/,https://www.reddit.com/r/PurgingDisorder/comme...,1630218867,127,0,0,0,-1,1
7673,24619,PurgingDisorder,i keep feeling the need to go into my bathroom...,NaN,help,3,NaN,NaN,NaN,NaN,...,NaN,/r/PurgingDisorder/comments/ol3ha5/help/,https://www.reddit.com/r/PurgingDisorder/comme...,1626387966,124,0,0,0,-1,1
7674,24624,PurgingDisorder,Sometimes I come to reddit to read about peopl...,NaN,So Conflicted...,6,NaN,NaN,NaN,NaN,...,NaN,/r/PurgingDisorder/comments/lrzfty/so_conflicted/,https://www.reddit.com/r/PurgingDisorder/comme...,1614232007,488,0,0,0,-1,1


In [1]:
import openai
from utils import *
import pandas as pd
narrative_df = pd.read_csv('../narrative_detection/narrative_posts_by_trained_classification.csv')
narrative_df

,Unnamed: 0,subreddit,selftext,author_fullname,title,score,link_flair_css_class,link_flair_text,author_flair_type,over_18,...,author_flair_css_class,permalink,url,created_utc,num_words,narrative_positive,narrative_negative,narrative_uncertain,narrative_label,predicted_narrative_label
0,0,EatingDisorders,23 F. I just started treatment for an eating d...,NaN,I need moral support,1,NaN,seeking support,NaN,NaN,...,NaN,/r/EatingDisorders/comments/10poqxu/i_need_mor...,https://www.reddit.com/r/EatingDisorders/comme...,1675140001,191,0,0,0,-1,1
1,1,EatingDisorders,I 26 f told the truth for the first time about...,NaN,Opening up about my ed,1,NaN,seeking support,NaN,NaN,...,NaN,/r/EatingDisorders/comments/10pnoxo/opening_up...,https://www.reddit.com/r/EatingDisorders/comme...,1675137504,273,0,0,0,-1,1
2,4,EatingDisorders,Idk who to talk to about this so I resort to r...,NaN,Just ranting ig,1,NaN,NaN,NaN,NaN,...,NaN,/r/EatingDisorders/comments/10pjinj/just_ranti...,https://www.reddit.com/r/EatingDisorders/comme...,1675126245,133,0,0,0,-1,1
3,5,EatingDisorders,I don’t know where to start.. Due to my upbrin...,NaN,Is true recovery possible?,1,NaN,seeking support,NaN,NaN,...,NaN,/r/EatingDisorders/comments/10pj90n/is_true_re...,https://www.reddit.com/r/EatingDisorders/comme...,1675125516,132,0,0,0,-1,1
4,8,EatingDisorders,i abused laxatives/purging/starving and now ev...,NaN,i get “chronic” hiccups due to bulimia (take t...,1,NaN,NaN,NaN,NaN,...,NaN,/r/EatingDisorders/comments/10perl4/i_get_chro...,https://www.reddit.com/r/EatingDisorders/comme...,1675114462,171,0,0,0,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7671,24609,bingeeating,I saw something like this posted on another we...,NaN,You might have a binge eating disorder if...,1,NaN,NaN,NaN,NaN,...,NaN,/r/bingeeating/comments/4iqjys/you_might_have_...,https://www.reddit.com/r/bingeeating/comments/...,1462903827,310,0,0,0,-1,1
7672,24617,PurgingDisorder,"hi,\n\nthanks for making this space.\n\ni was ...",NaN,hi,6,NaN,NaN,NaN,NaN,...,NaN,/r/PurgingDisorder/comments/pdq9pz/hi/,https://www.reddit.com/r/PurgingDisorder/comme...,1630218867,127,0,0,0,-1,1
7673,24619,PurgingDisorder,i keep feeling the need to go into my bathroom...,NaN,help,3,NaN,NaN,NaN,NaN,...,NaN,/r/PurgingDisorder/comments/ol3ha5/help/,https://www.reddit.com/r/PurgingDisorder/comme...,1626387966,124,0,0,0,-1,1
7674,24624,PurgingDisorder,Sometimes I come to reddit to read about peopl...,NaN,So Conflicted...,6,NaN,NaN,NaN,NaN,...,NaN,/r/PurgingDisorder/comments/lrzfty/so_conflicted/,https://www.reddit.com/r/PurgingDisorder/comme...,1614232007,488,0,0,0,-1,1


In [11]:
last_narratives = list(narrative_df.tail(5).selftext)
prompts = []
for narrative in last_narratives:
    prompts.append(get_few_shot_examples() + 'Narrative: ' + narrative + '\n\n')


In [12]:
narrative

"I'm overweight because I love binge eating. But sometimes I hate that I binge eat so I purge. But sometimes I love purging so I don't even have to binge to do it. But then I hate that I'm purging so I restrict. But I can't restrict forever and I hate feeling full so I start restricting and purging. Except those days where I eat relatively normally, but now I have energy so I start exercising. And I can handle more and more every day. And I'm making progress, but not enough.\n\nSo I start restricting again. But I'm still exercising. But I still feel too full so I start purging again, even though I'm still exercising. And after a while I have a day where I'm so starved that I binge. And then purge. And the cycle continues.\n\nAnd I start to eat things that aren't food. I'm eating my fingernails, bits of paper (hardback book paper tastes best), pen caps... And I purge that too, because anything being in your stomach for too long is obviously poison. \n\nI'm just struggling and ranting. S

In [15]:
prompts

["\n    Here are some examples: \n\n    Narrative: i abused laxatives/purging/starving and now every single time i eat i will get the hiccups, this can be anywhere from 2 minutes to 7 whole hours. its making recovery SO hard. the involuntary movements hurt my ribcage, it feels like i'm being punished for eating. i did it to myself. recovery feels useless now. pls don't get to the point i'm at because you will regret it and think back at every day of your life when you didn't have this. i can't play guitar properly anymore if i eat bc i will constantly hiccup against my guitar, my hiccups move my whole chest. it makes any form of exercise hard. people who don't know my hiccups are chronic laugh when i get them and try to scare them away as a joke etc. this is not the life you want to live. this is both a vent and a warning post. any supportive messages are welcome i'm going through a really rough time so please keep anything nasty to yourself\n\n    {'factors': 'Abusing laxatives, purgi

In [3]:
len(prompts)

5

In [8]:
response = openai.Completion.create(
    model="davinci",
    prompt=prompts,
    max_tokens= 70,
    temperature=0.1
)

In [13]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    n = 3,
    messages=[
            {"role": "user", "content":get_new_instruction_prompt()},
            {"role": "user", "content":get_few_shot_examples()},
            {"role": "user", "content": "Narrative: " + narrative}])

In [15]:
response.choices

[<OpenAIObject at 0x12ed1adb0> JSON: {
   "finish_reason": "stop",
   "index": 0,
   "message": {
     "content": "{'factors': 'Binge eating, purging, restricting, and exercising', 'effect_details': ['Engaging in an unhealthy cycle of binging, purging, restricting, and exercising', 'Eating non-food items like fingernails and paper', 'Believing that anything in the stomach for too long is poisonous'], 'effect_type': 'harmful'}",
     "role": "assistant"
   }
 },
 <OpenAIObject at 0x12ed1a540> JSON: {
   "finish_reason": "stop",
   "index": 1,
   "message": {
     "content": "{'factors': 'Binge eating, purging, restricting, and exercising', 'effect_details': ['Cyclical pattern of disordered eating behaviors', 'Eating non-food items', 'Struggling with progress', 'Feeling like anything in stomach for too long is poison'], 'effect_type': 'harmful'}",
     "role": "assistant"
   }
 },
 <OpenAIObject at 0x12ed1acc0> JSON: {
   "finish_reason": "stop",
   "index": 2,
   "message": {
     "cont

In [46]:

def apply_chatgpt(row, prompt):
    narrative = row['selftext']
    max_retries = 3
    first_time_fail = False
    for i in range(1, 3):
        retries = 0
        while retries < max_retries:
            try:
                if first_time_fail: 
                    row[f'perspective_{i}'] = None
                    row[f'treatment_{i}'] = None
                    row[f'effect_type_{i}'] = None
                    row[f'effect_details_{i}'] = None
                    return row
                # result = eval(result)
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[
                            {"role": "user", "content":prompt},
                            {"role": "user", "content": narrative},])
                result = response.choices[0].message.content
                result = eval(result)
                row['perspective'] = result['perspective']
                row['treatment'] = result['treatment']
                row['effect_type'] = result['effect_type']
                row['effect_details'] = result['effect_details']
                break
            except Exception as e:
                retries += 1
                if retries == max_retries:
                    row[f'perspective_{i}'] = None
                    row[f'treatment_{i}'] = None
                    row[f'effect_type_{i}'] = None
                    row[f'effect_details_{i}'] = None
                    first_time_fail = True
    return row

In [47]:
changes = test_df.apply(apply_chatgpt, axis=1, prompt='hi')
print(changes.columns)
changes.head()


Index(['author', 'subreddit_id', 'selftext', 'created_utc', 'link_flair_text',
       'title', 'perspective_1', 'treatment_1', 'effect_type_1',
       'effect_details_1', 'perspective_2', 'treatment_2', 'effect_type_2',
       'effect_details_2'],
      dtype='object')


,author,subreddit_id,selftext,created_utc,link_flair_text,title,perspective_1,treatment_1,effect_type_1,effect_details_1,perspective_2,treatment_2,effect_type_2,effect_details_2
0,--UsernameNotFound--,t5_2zzq8,['I struggling whit body image and every time ...,[1609984721],[nan],['Can this be an Ed or just life'],1,1,1,1,2,2,2,2
1,-CreamyPie-,t5_o3plh,['Hello!\n\nIm 11 and Im a boy self recovering...,"[1588343070, 1588190127, 1587836033]","['recovery win', nan, nan]","['Fear Food Friday!! So far its successful', '...",1,1,1,1,2,2,2,2
2,-Hawkeye,t5_o3plh,['My ex has disclosed to me that she goes thro...,[1612196819],[nan],"[""Need Advice with Ex's Disorder""]",1,1,1,1,2,2,2,2
3,-Perchta-,t5_2zzq8,"['So I think I may have an eating disorder, bu...",[1569294268],[nan],['I need help with ways to keep food down'],1,1,1,1,2,2,2,2
4,-buppy-,t5_2tmc8,['I (F 18) have always been chubbier. While I ...,[1670794694],[nan],"['I know I need help, but I don’t want it.']",1,1,1,1,2,2,2,2


In [31]:
test_df.columns

Index(['author', 'subreddit_id', 'selftext', 'created_utc', 'link_flair_text',
       'title'],
      dtype='object')

In [17]:
import openai
import os

instruction_prompt = """I am going to be giving you narratives about people with eating disorders. 
Can you help me identify what helped the user with their eating disorder or what made their eating disorder worse?

I want you to first identify the person that they are writing about. If it's a first person narrative then the person would be the writer,
if it's unknown then say that it's the writer, else if it's second or third person please identify that person.

Then, I want you to identify what has helped or hurt their eating disorder, this will be the treatment. If it's multiple treatments
I want you to provide the answer as a list of strings, else I just want it to be a string. 

I also want you to explain whether that treatment on a person was helpful to the person or harmful for them. This will be the effect. 

The outcome can only be helpful, harmful, or neutral. If the treatment was helpful then it helped the person recover, if it was harmful then it harmed the user
and affected their recovery, if it was neutral it didn't do anything. 

If the narrative isn't related to an eating disorder please just return None for perspective, treatment, effect, and outcome. 

Then I want you to send that answer to me.
IMPORTANT: ONLY SEND ME THE FORMATTED ANSWER.

want the answer to be formatted like this so it forms a python dictionary:
{'perspective': '[PERSON IDENTIFIED HERE]', 
'treatment': '[WHAT HELPED OR HARMED THEM HERE]', 
'effect': '[EXPLAIN THE EFFECT THE TREATMENT HAS HAD ON THE WRITER]', 
'outcome': '[WHETHER THE TREATMENT WAS HELPFUL OR HARMFUL]'}
"""


def apply_chatgpt(row, prompt):
  narrative = row['selftext']
  max_retries = 2
  retries = 0
  while retries < max_retries:
    try:
      response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            {"role": "user", "content":prompt},
            {"role": "user", "content": narrative},])

      result = response.choices[0].message.content
      print(result)
      result = eval(result)
      row['perspective'] = result['perspective']
      row['treatment'] = result['treatment']
      row['effect_type'] = result['effect_type']
      row['effect_details'] = result['effect_details']
      return row
    except Exception as e:
        retries += 1
        if retries == max_retries:
          row['perspective'] = None
          row['treatment'] = None
          row['effect_type'] = None
          row['effect_details'] = None
          return row

In [18]:
instruction_df = positive_df.copy(deep=True).reset_index().head(20)
# instruction_df = instruction_df.head(20)
instruction_df = instruction_df.apply(apply_chatgpt, axis=1, prompt=instruction_prompt)


{'perspective': 'writer', 
'treatment': 'Smoothies', 
'effect': 'Smoothies helped the writer in their recovery', 
'outcome': 'helpful'}
{'perspective': 'writer', 
'treatment': 'Smoothies', 
'effect': 'helped me in my recovery', 
'outcome': 'helpful'}
{'perspective': 'writer', 
'treatment': 'Improving their eating habits by incorporating more meals and snacks',
'effect': 'The improved eating habits have led to an increase in energy and elimination of headaches, causing the writer to feel proud of their progress.',
'outcome': 'Helpful'}
{'perspective': 'Writer', 
'treatment': 'Increase in meals and snacks', 
'effect': 'Increasing meals and snacks in the diet was helpful for the writer as it has helped them gain more energy and reduce headaches.', 
'outcome': 'Helpful'}
{'perspective': 'Writer', 
'treatment': 'Unknown', 
'effect': 'Unknown', 
'outcome': 'Unknown'}
{'perspective': 'writer', 
'treatment': 'unknown', 
'effect': 'unknown', 
'outcome': 'unknown'}
{'perspective': 'writer', 
'tr

In [20]:
instruction_df['perspective'].value_counts()

Series([], Name: perspective, dtype: int64)

In [68]:
automatic_prompt = """
Please analyze the following narrative related to eating disorders and extract the relationships between specific clinical treatments and their positive or negative effects on the individual. 
Focus on identifying pharmaceutical treatments and their impact. For each narrative, provide a summary in the following format: {'perspective': ..., 'treatment': '...' or ['...', '...'], 'effect_type': harmful/helpful/neutral/unknown, 'effect_details': '...'}

'perspective': This refers to the point of view or the person speaking in the narrative. In most cases, it would be 'the writer' as it is their personal experience being shared.

'treatment': This refers to the specific treatment(s) or intervention(s) used by the individual, with a focus on pharmaceutical or clinical treatments. Examples include medications, therapy, or other clinical interventions. 
If only one treatment is mentioned in the narrative, the 'treatment' will be a single string value. If multiple treatments are discussed, 'treatment' will be a list of strings, each representing a different treatment.

'effect_type': For outcome here are the different options to choose from: 	
1. harmful': This outcome indicates that the treatment had a negative impact on the individual's eating disorder or well-being. It may have worsened their condition, led to adverse side effects, or created additional problems.
2. 'helpful': This outcome suggests that the treatment had a positive effect on the individual's eating disorder or overall well-being. It may have improved their condition, helped them manage their symptoms, or led to other beneficial outcomes.
3. 'neutral': This outcome means that the treatment neither helped nor harmed the individual's eating disorder or well-being. It could imply that the treatment had no noticeable effect or that any positive and negative effects balanced each other out.
4. 'unknown': This outcome is used when it's unclear or not explicitly mentioned in the narrative whether the treatment had a positive, negative, or neutral effect on the individual's eating disorder or well-being.

'effect_details': This provides a brief description of the specific effects or consequences the treatment had on the individual's life or eating disorder. It may include improvements or setbacks in their mental health, changes in their eating habits, or other related outcomes.
"""

automatic_prompt_df = positive_df.copy(deep=True).reset_index()
automatic_prompt_df = automatic_prompt_df.apply(apply_chatgpt, axis=1, prompt=automatic_prompt)
# automatic_prompt_df.to_csv('automatic_prompt.csv')


In [3]:
import pandas as pd
automatic_prompt_df = pd.read_csv('automatic_prompt.csv')

In [22]:
instruction_df = pd.read_csv('instruction_df.csv')

In [23]:
instruction_df.head(20)

,Unnamed: 0,index,title,link_flair_text,selftext,perspective,treatment,effect,outcome
0,0,13,ED tip if you are trying to eat more,recovery story,Smoothies really helped me. You can put a bana...,writer,Smoothies,Smoothies helped the writer in their recovery,helpful
1,1,58,Recovery Story,recovery story,\n\nI started getting insecure during lockdown...,Writer,Recovering from unhealthy eating habits by add...,Adding more meals and snacks to their daily ro...,Helpful
2,2,109,This has been going on since 2006,recovery story,17 years of ED. It just seems like this will b...,writer,NaN,NaN,NaN
3,3,143,so proud of myself,recovery story,I'm six days binge free! Today i didn't rexist...,writer,unknown,The writer seems to be proud of themselves and...,neutral
4,4,154,It gets better,recovery story,I'm more than 2 years into recovery. I promise...,writer,unknown,The writer has successfully recovered from the...,helpful
5,5,316,getting better :),recovery story,I've been managing to eat full meals for a few...,Writer,"Eating meals until full, friend bringing homem...",Writer is feeling happy and reassured by the f...,Helpful
6,6,325,just fully finished my first complete meal in ...,recovery story,i've been better at eating more nutritious foo...,writer,eating more nutritious food,"Helpful, but still struggling with proper meals",Neutral
7,7,483,Eating disorder win!,recovery story,I have a habit of eating until my plate is cle...,writer,['Ate until half-full instead of finishing the...,This helped the person feel better and more ho...,helpful
8,8,489,I ate a whole bag of crisps,recovery story,It was definitely a milestone in my recovery j...,writer,unknown,unknown,unknown
9,9,7171,Can I ask you guys' advice on something?,story time,I have this college friend who's 4 years young...,Writer,Unknown,The conversation with her friend was triggerin...,Neutral


In [24]:
instruction_df.perspective.value_counts()

Writer          74
writer          68
the writer       4
first            1
Unknown          1
first person     1
The writer       1
Olyguy (M48)     1
Name: perspective, dtype: int64

In [6]:
automatic_prompt_df.shape

(159, 9)

In [8]:
automatic_prompt_df.treatment.unique()

array(['smoothies', nan, 'unknown', 'none', 'Prozac',
       'recovery (not specified)', 'weight gain',
       '[\'forcing myself to eat "unclean" food", exercise\', \'being patient\']',
       'not specified',
       "['recovery', 'extreme hunger', 'going to the gym']", 'therapy',
       'recovery regimen', 'Rybelsus', 'Concerta for ADHD',
       'no treatment mentioned', 'Vyvanse'], dtype=object)

In [9]:
automatic_prompt_df['treatment'].head(20)

0     smoothies
1           NaN
2       unknown
3           NaN
4           NaN
5           NaN
6           NaN
7           NaN
8           NaN
9           NaN
10          NaN
11          NaN
12          NaN
13          NaN
14          NaN
15          NaN
16          NaN
17          NaN
18          NaN
19          NaN
Name: treatment, dtype: object

In [5]:
automatic_prompt_df['treatment'].value_counts()

unknown                                                                 3
none                                                                    3
smoothies                                                               1
Prozac                                                                  1
recovery (not specified)                                                1
weight gain                                                             1
['forcing myself to eat "unclean" food", exercise', 'being patient']    1
not specified                                                           1
['recovery', 'extreme hunger', 'going to the gym']                      1
therapy                                                                 1
recovery regimen                                                        1
Rybelsus                                                                1
Concerta for ADHD                                                       1
no treatment mentioned                

In [63]:
automatic_prompt_df.head()

,index,title,link_flair_text,selftext,perspective,treatment,effect_type,effect_details
0,13,ED tip if you are trying to eat more,recovery story,Smoothies really helped me. You can put a bana...,the writer,Smoothies,helpful,Improved the writer’s eating habits and aided ...
1,58,Recovery Story,recovery story,\n\nI started getting insecure during lockdown...,None,None,None,None
2,109,This has been going on since 2006,recovery story,17 years of ED. It just seems like this will b...,the writer,unknown,unknown,No specific clinical treatments or interventio...
3,143,so proud of myself,recovery story,I'm six days binge free! Today i didn't rexist...,None,None,None,None
4,154,It gets better,recovery story,I'm more than 2 years into recovery. I promise...,None,None,None,None


In [51]:
# instruction_df.to_csv('instruction_df.csv')

In [25]:
row = instruction_df.iloc[0]
narrative = row['selftext']
response = openai.ChatCompletion.create(
model="gpt-3.5-turbo",
messages=[
        {"role": "user", "content":instruction_prompt},
        {"role": "user", "content": narrative},
    ]
)

result = eval(response.choices[0].message.content)
row['perspective'] = result['perspective']
row['treatment'] = result['treatment']
row['effect'] = result['effect']
row['outcome'] = result['outcome']

/var/folders/32/km91km0x2s98wl2f_rb4gb0r0000gn/T/ipykernel_62684/2995633047.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['perspective'] = result['perspective']
/var/folders/32/km91km0x2s98wl2f_rb4gb0r0000gn/T/ipykernel_62684/2995633047.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['perspective'] = result['perspective']
/var/folders/32/km91km0x2s98wl2f_rb4gb0r0000gn/T/ipykernel_62684/2995633047.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [26]:
result

{'perspective': 'writer',
 'treatment': 'Smoothies',
 'effect': 'Smoothies helped the writer with their recovery from the eating disorder.',
 'outcome': 'Positive'}

In [33]:
eval(response['choices'][0]['message']['content'])

{'perspective': 'Writer',
 'treatment': 'Smoothies',
 'effect': 'Smoothies helped the writer with their eating disorder recovery.'}

'The 2020 World Series was played at Globe Life Field in Arlington, Texas.'

In [6]:
openai.Model.list()


<OpenAIObject list at 0x1344a99f0> JSON: {
  "data": [
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1669085501,
          "group": null,
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "babbage"
    },
    {
      "created": 1649359874,
      "id": "davinci",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampl